In [ ]:
## This is the answer guide notebook 
## It has the un-modified data and good hyperparams

In [26]:
# TODO Get this made public
! pip install git+https://github.com/deepskies/DeepBench.git


  Cloning https://github.com/deepskies/DeepBench.git (to revision bug/import) to /private/var/folders/95/_pswfcf9723cb9f4vw6h944m0000gr/T/pip-req-build-gt72anhg
  Running command git clone --filter=blob:none --quiet https://github.com/deepskies/DeepBench.git /private/var/folders/95/_pswfcf9723cb9f4vw6h944m0000gr/T/pip-req-build-gt72anhg
  Running command git checkout -b bug/import --track origin/bug/import
  Switched to a new branch 'bug/import'
  branch 'bug/import' set up to track 'origin/bug/import'.
  Resolved https://github.com/deepskies/DeepBench.git to commit 2718040cd0fd0d51b78146f42cbe684d70f103b1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for deepbench: filename=deepbench-0.1.1-py3-none-any.whl size=29920 sha256=ce11d8beb7c05e43dbb220d1a7762fee9ae23f18942247874ac0443cc11d91f4
  Stored in directory: /private/var/folders/95/_pswfcf9723cb9f4vw6h944m0000gr/T/pip-ephem-wheel

In [97]:
import numpy as np
import matplotlib.pyplot as plt
from deepbench.astro_object import StarObject, GalaxyObject 

from tensorflow.keras.layers import Input, Dropout, Conv1D, Dense, AvgPool1D, Flatten
from tensorflow.keras.models import Model

import tensorflow as tf 

import math 


In [120]:
# don't worry too much about how this simulation is being run. It's fine. Don't worry. I promise :3c 

# This generates data as per the Secret Parameters 
# I request you don't change anything here or look too closely. 
# Otherwise! this challenge won't be as fun

class SkyGenerator(tf.keras.utils.Sequence): 
    def __init__(self, n_samples, batch_size=64, pre_processing=None, train=True, shuffle=False):
        self.n_samples = n_samples
        if train: 
            self.n_samples = math.ceil(0.25*n_samples)

        self.pre_processing = pre_processing

        self.batch_size = batch_size
        self.shuffle = shuffle

        self.image_size = 14
        self.noise_level = 0.05

        self.rng = np.random.default_rng(seed=42) # Seed for the main notebook

        self.labels = self.decide_labels()

    def decide_labels(self): 
        n_stars = self.rng.integers(low=int(.45*self.n_samples), high=int(.65*self.n_samples))
        n_galaxies = self.n_samples-n_stars
        labels = [0 for _ in range(n_stars)] + [1 for _ in range(n_galaxies)]
        if self.shuffle: 
            self.rng.shuffle(labels)
        return np.asarray(labels)
 
    def generate_image(self, label): 
        radius = self.rng.integers(low=1, high=self.image_size/2)
        center_x = self.rng.integers(low=1, high=self.image_size)
        center_y = self.rng.integers(low=1, high=self.image_size)

        if label == 0: 
            image = StarObject(
                image_dimensions=self.image_size,
                noise=self.noise_level,
                radius=radius
                    ).create_object(
                        center_x=center_x, center_y=center_y
                        )
 
        else: 
            image = GalaxyObject(
                image_dimensions=self.image_size,
                noise_level=self.noise_level,
                radius=radius
                    ).create_object(
                        center_x=center_x, center_y=center_y
                        )

        if self.pre_processing is not None: 
            image = self.pre_processing.transform(image)

        return image

    def __len__(self):
        return math.ceil(self.n_samples / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        high = min(low + self.batch_size, len(self.labels))
        batch_y = self.labels[low:high]
        batch_x = np.zeros((len(batch_y), self.image_size, self.image_size))
        for index, label in enumerate(batch_y): 
            batch_x[index] = self.generate_image(label)

        return batch_x, batch_y

In [121]:
def make_model(): 
    in_layer = Input((14, 14))
    x = Conv1D(filters=4, kernel_size=2)(in_layer)
    x = Conv1D(filters=8, kernel_size=4)(x)
    x = Conv1D(filters=12, kernel_size=6)(x)
    
    x = AvgPool1D(4)(x)

    x = Flatten()(x)
    x = Dense(200, activation='relu')(x)
    x = Dense(10, activation='relu')(x)

    x = Dropout(0.3)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(in_layer, output)
    
    return model

In [122]:
model = make_model()
model.compile(loss='binary_crossentropy', optimizer="SGD")
model.summary()

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 14, 14)]          0         
                                                                 
 conv1d_76 (Conv1D)          (None, 13, 4)             116       
                                                                 
 conv1d_77 (Conv1D)          (None, 10, 8)             136       
                                                                 
 conv1d_78 (Conv1D)          (None, 5, 12)             588       
                                                                 
 average_pooling1d_23 (Avera  (None, 1, 12)            0         
 gePooling1D)                                                    
                                                                 
 flatten_20 (Flatten)        (None, 12)                0         
                                                          

In [125]:
train_generator = SkyGenerator(n_samples=1280, shuffle=True)
val_generator = SkyGenerator(n_samples=1280, train=False, shuffle=True)
history = model.fit(
    train_generator, 
    validation_data=val_generator, 
    epochs=20, 
    verbose=1
    ).history


Epoch 1/20
3/5 [=================>............] - ETA: 0s - loss: 0.6626

2023-06-16 12:27:55.553266: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 175ms/step - loss: 0.6604 - val_loss: 0.6552
Epoch 2/20
5/5 [==============================] - 1s 141ms/step - loss: 0.6495 - val_loss: 0.6404
Epoch 3/20
5/5 [==============================] - 1s 144ms/step - loss: 0.6394 - val_loss: 0.6271
Epoch 4/20
5/5 [==============================] - 1s 144ms/step - loss: 0.6241 - val_loss: 0.6161
Epoch 5/20
5/5 [==============================] - 1s 141ms/step - loss: 0.6094 - val_loss: 0.6061
Epoch 6/20
5/5 [==============================] - 1s 141ms/step - loss: 0.6082 - val_loss: 0.5979
Epoch 7/20
5/5 [==============================] - 1s 141ms/step - loss: 0.6012 - val_loss: 0.5880
Epoch 8/20
5/5 [==============================] - 1s 143ms/step - loss: 0.5888 - val_loss: 0.5826
Epoch 9/20
5/5 [==============================] - 1s 143ms/step - loss: 0.5766 - val_loss: 0.5670
Epoch 10/20
5/5 [==============================] - 1s 142ms/step - loss: 0.5744 - val_loss: 0.5641
Epoch 11/20
5/5 [=============

In [48]:
# Plot history


In [ ]:
# Making a different testing set (So people can see the in-domain vs out-of-domain)
testing_set = {}
testing_generator = {}

In [ ]:
# Metrics and evaluation